# TD - Graph Traversal: BFS, DFS and Shortest Paths

## Objectives
- Understand and implement breadth-first search (BFS)
- Understand and implement depth-first search (DFS)
- Use shortest path algorithms
- Visualize results with Plotly on a real graph (Paris road network)

## Data
We will use the Paris road network obtained via OSMnx, which retrieves OpenStreetMap data.


In [1]:
# Installation of necessary libraries
# !pip install osmnx networkx plotly matplotlib numpy

In [2]:
import osmnx as ox
import networkx as nx
import plotly.graph_objects as go
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from IPython.display import display, HTML

print("Libraries imported successfully!")
print(f"NetworkX version: {nx.__version__}")
print(f"OSMnx version: {ox.__version__}")

Libraries imported successfully!
NetworkX version: 3.2.1
OSMnx version: 2.0.6


## 1. Loading the Paris Road Network Graph


In [3]:
# Loading the Paris road network graph
print("Loading Paris road network...")
G = ox.graph_from_place("Paris, France", network_type="drive")

# Convert to undirected NetworkX graph to simplify algorithms
# (we can keep the directed graph if necessary)
G = G.to_undirected()

print(f"Graph loaded: {len(G.nodes())} nodes, {len(G.edges())} edges")
print(f"Graph type: {type(G)}")
print(f"Is connected: {nx.is_connected(G)}")

Loading Paris road network...
Graph loaded: 9434 nodes, 14768 edges
Graph type: <class 'networkx.classes.multigraph.MultiGraph'>
Is connected: True


In [4]:
# Select a main connected component if necessary
if not nx.is_connected(G):
    print("Graph is not connected. Selecting the largest connected component...")
    largest_cc = max(nx.connected_components(G), key=len)
    G = G.subgraph(largest_cc).copy()
    print(f"New graph: {len(G.nodes())} nodes, {len(G.edges())} edges")

# Select two nodes for our experiments
nodes = list(G.nodes())
start_node = nodes[0]
end_node = nodes[len(nodes)//4]  # A node at approximately 1/4 of the graph

print(f"\nStart node: {start_node}")
print(f"End node: {end_node}")


Start node: 125730
End node: 27235559


## 2. Utility Function for Visualization with Plotly


In [5]:
def plot_graph_plotly(G, title="Graph", highlight_nodes=None, highlight_path=None, node_colors=None):
    """
    Visualize a graph with Plotly on a map
    
    Parameters:
    - G: NetworkX graph
    - title: chart title
    - highlight_nodes: list of nodes to highlight
    - highlight_path: list of nodes forming a path to highlight
    - node_colors: dictionary {node: color} to color nodes
    """
    # Get coordinates (longitude, latitude)
    pos = {}
    for node in G.nodes():
        if 'x' in G.nodes[node] and 'y' in G.nodes[node]:
            # OSMnx stores x=longitude, y=latitude
            pos[node] = (G.nodes[node]['x'], G.nodes[node]['y'])
        elif 'lon' in G.nodes[node] and 'lat' in G.nodes[node]:
            pos[node] = (G.nodes[node]['lon'], G.nodes[node]['lat'])
        else:
            # Fallback: use node ID as coordinates
            pos[node] = (node, 0)
    
    # Calculate map center
    if pos:
        lons = [p[0] for p in pos.values() if isinstance(p[0], (int, float))]
        lats = [p[1] for p in pos.values() if isinstance(p[1], (int, float))]
        if lons and lats:
            center_lon = sum(lons) / len(lons)
            center_lat = sum(lats) / len(lats)
        else:
            center_lon, center_lat = 2.3522, 48.8566  # Paris by default
    else:
        center_lon, center_lat = 2.3522, 48.8566
    
    fig = go.Figure()
    
    # Prepare edges with coordinates (lon, lat)
    for edge in G.edges():
        if edge[0] in pos and edge[1] in pos:
            lon0, lat0 = pos[edge[0]]
            lon1, lat1 = pos[edge[1]]
            # Check that coordinates are valid
            if (isinstance(lon0, (int, float)) and isinstance(lat0, (int, float)) and
                isinstance(lon1, (int, float)) and isinstance(lat1, (int, float))):
                fig.add_trace(go.Scattermap(
                    mode='lines',
                    lon=[lon0, lon1],
                    lat=[lat0, lat1],
                    line=dict(width=0.5, color='#888'),
                    hoverinfo='none',
                    showlegend=False
                ))
    
    # Prepare nodes
    node_lons = []
    node_lats = []
    node_text = []
    node_colors_list = []
    node_sizes = []
    
    for node in G.nodes():
        if node in pos:
            lon, lat = pos[node]
            if isinstance(lon, (int, float)) and isinstance(lat, (int, float)):
                node_lons.append(lon)
                node_lats.append(lat)
                node_text.append(f'Node {node}')
                
                # Determine color and size
                if highlight_path and node in highlight_path:
                    color = 'red'
                    size = 10
                elif highlight_nodes and node in highlight_nodes:
                    color = 'orange'
                    size = 12
                elif node_colors and node in node_colors:
                    color = node_colors[node]
                    size = 8
                else:
                    color = 'lightblue'
                    size = 4
                node_colors_list.append(color)
                node_sizes.append(size)
    
    # Add trace for nodes
    if node_lons and node_lats:
        fig.add_trace(go.Scattermap(
            mode='markers',
            lon=node_lons,
            lat=node_lats,
            text=node_text,
            hoverinfo='text',
            marker=dict(
                size=node_sizes,
                color=node_colors_list,
                opacity=0.8
            ),
            showlegend=False
        ))
    
    # Update layout with mapbox
    fig.update_layout(
        title=title,
        mapbox=dict(
            style="open-street-map",  # Use OpenStreetMap as basemap
            center=dict(lon=center_lon, lat=center_lat),
            zoom=11,  # Zoom level adapted to Paris
            bearing=0,
            pitch=0
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        height=700
    )
    
    return fig

## 3. Initial Graph Visualization


In [6]:
# Visualization of the complete graph (sampled for performance)
sample_nodes = list(G.nodes())[::10]  # Take 1 node out of 10 for visualization
G_sample = G.subgraph(sample_nodes).copy()

fig = plot_graph_plotly(G_sample, title="Paris Road Network (sample)")
fig.show()


## 4. Breadth-First Search (BFS)

### 4.0 BFS Theory

**Breadth-First Search (BFS)** is a fundamental graph traversal algorithm that systematically explores a graph level by level.

#### Principle of Operation

BFS uses a **queue (FIFO: First In First Out)** to maintain the order of node visits:

1. **Initialization**: Start with a source node, mark it as visited and add it to the queue
2. **Exploration**: While the queue is not empty:
   - Remove the first node from the queue (dequeue)
   - Visit all its unvisited neighbors
   - Mark these neighbors as visited and add them to the end of the queue (enqueue)
3. **Result**: We obtain the order of visits level by level

#### Important Properties

- **Shortest path**: On an **unweighted** graph, BFS guarantees finding the shortest path in number of edges between two nodes
- **Data structure**: Queue (FIFO) - the first node added is the first to be processed
- **Time complexity**: O(V + E) where V is the number of nodes and E is the number of edges
- **Space complexity**: O(V) to store the queue and visited nodes

#### Applications

- Maze navigation
- Social networks: finding the shortest path between two users
- Network routing: finding the path with the least hops
- Graph connectivity verification
- Cycle detection (on undirected graphs)

#### Pseudocode

```
BFS(G, start):
    queue = [start]
    visited = {start}
    order = [start]
    parent = {start: None}
    
    while queue not empty:
        node = queue.dequeue()
        
        for neighbor in G.neighbors(node):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.enqueue(neighbor)
                order.append(neighbor)
                parent[neighbor] = node
    
    return visited, order, parent
```

#### Visual Example

For a graph with starting node A:
- Level 0: A
- Level 1: all direct neighbors of A
- Level 2: all neighbors of nodes at level 1 (not already visited)
- etc.

### 4.1 BFS Implementation


In [7]:
def bfs(G, start, end=None):
    """
    Breadth-first search (BFS)
    
    Parameters:
    - G: NetworkX graph
    - start: starting node
    - end: destination node (optional, if None, traverses entire graph)
    
    Returns:
    - visited: set of visited nodes
    - path: path to 'end' if specified, None otherwise
    - order: order of node visits
    """
    visited = set()
    queue = deque([start])
    visited.add(start)
    order = [start]
    parent = {start: None}
    
    while queue:
        node = queue.popleft()
        
        # If we're looking for a specific node and found it
        if end is not None and node == end:
            # Reconstruct the path
            path = []
            current = end
            while current is not None:
                path.append(current)
                current = parent[current]
            return visited, list(reversed(path)), order
        
        # Traverse neighbors
        for neighbor in G.neighbors(node):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
                parent[neighbor] = node
                order.append(neighbor)
    
    # If end was specified but not found
    if end is not None:
        return visited, None, order
    
    return visited, None, order

# Test BFS
print("Running BFS...")
visited_bfs, path_bfs, order_bfs = bfs(G, start_node, end_node)

print(f"Number of visited nodes: {len(visited_bfs)}")
print(f"Path length found: {len(path_bfs) if path_bfs else 'No path'}")

if path_bfs:
    print(f"Path (first 10 nodes): {path_bfs[:10]}...")
    print(f"Path (last 10 nodes): ...{path_bfs[-10:]}")


Running BFS...
Number of visited nodes: 3774
Path length found: 37
Path (first 10 nodes): [125730, 12179625841, 4467439444, 21112878, 331861643, 76291820, 7475430200, 84791191, 25245056, 25245072]...
Path (last 10 nodes): ...[209165524, 25075891, 255028808, 27286561, 27235582, 442309, 27235547, 27235540, 27235565, 27235559]


### 4.2 BFS Visualization


In [8]:
# Create a subgraph containing nodes visited by BFS
if path_bfs:
    nodes_in_path = set()
    for i in range(len(path_bfs) - 1):
        nodes_in_path.add(path_bfs[i])
        nodes_in_path.add(path_bfs[i+1])
        # Add immediate neighbors to see the context
        for neighbor in G.neighbors(path_bfs[i]):
            nodes_in_path.add(neighbor)

    G_bfs = G.subgraph(list(nodes_in_path)[:500]).copy()  # Limit to 500 nodes for performance

    # Create a color dictionary to show visit order
    node_colors = {}
    for i, node in enumerate(order_bfs[:100]):  # Limit for performance
        # Use a color palette
        if node in path_bfs:
            node_colors[node] = 'red'
        else:
            # Color based on visit order
            intensity = int(255 * (i / min(100, len(order_bfs))))
            node_colors[node] = f'rgb({intensity}, {255-intensity}, 200)'

    fig = plot_graph_plotly(
        G_bfs, 
        title="BFS Traversal - Paris Road Network",
        highlight_nodes=[start_node, end_node],
        highlight_path=path_bfs,
        node_colors=node_colors
    )
    fig.show()

    print(f"BFS visualization: {len(G_bfs.nodes())} nodes displayed")
else:
    print("No path found with BFS, cannot visualize")


BFS visualization: 78 nodes displayed


### 4.3 BFS with NetworkX (built-in function)


In [9]:
# Using NetworkX's built-in BFS function
try:
    bfs_edges_nx = list(nx.bfs_edges(G, source=start_node))
    bfs_tree_nx = nx.bfs_tree(G, source=start_node)
    
    print(f"NetworkX BFS - Number of edges explored: {len(bfs_edges_nx)}")
    print(f"NetworkX BFS - Tree size: {len(bfs_tree_nx.nodes())} nodes")
    
    # Shortest path with BFS (if weights are equal)
    try:
        path_bfs_nx = nx.shortest_path(G, source=start_node, target=end_node, method='bfs')
        print(f"NetworkX BFS Path - Length: {len(path_bfs_nx)}")
    except nx.NetworkXNoPath:
        print("No path found with NetworkX BFS")
        
except Exception as e:
    print(f"Error with NetworkX BFS: {e}")


NetworkX BFS - Number of edges explored: 9433
NetworkX BFS - Tree size: 9434 nodes
Error with NetworkX BFS: method not supported: bfs


## 5. Depth-First Search (DFS)

### 5.0 DFS Theory

**Depth-First Search (DFS)** is a graph traversal algorithm that explores the graph by going as far as possible along a branch before backtracking.

#### Principle of Operation

DFS uses a **stack (LIFO: Last In First Out)** or **recursion** to maintain the order of visits:

1. **Initialization**: Start with a source node, mark it as visited
2. **Recursive exploration**: For each visited node:
   - Recursively explore the first unvisited neighbor
   - Continue until reaching a node with no unvisited neighbors
   - Backtrack to explore other branches
3. **Result**: We obtain a depth-first traversal of the graph

#### Two Main Implementations

1. **Recursive DFS**: Uses the function call stack
2. **Iterative DFS**: Explicitly uses a data stack

#### Important Properties

- **Shortest path**: DFS **does NOT guarantee** finding the shortest path
- **Data structure**: Stack (LIFO) - the last node added is the first to be processed
- **Time complexity**: O(V + E) where V is the number of nodes and E is the number of edges
- **Space complexity**: O(V) for the stack (recursion or explicit stack) and visited nodes

#### Applications

- Cycle detection in a graph
- Topological sort (on directed acyclic graphs)
- Backtracking problem solving (mazes, sudoku)
- Finding strongly connected components
- Tree traversal (preorder, postorder)
- Graph algorithms (Hamiltonian path, etc.)

#### Pseudocode (recursive)

```
DFS_recursive(G, node, visited):
    visited.add(node)
    
    for neighbor in G.neighbors(node):
        if neighbor not in visited:
            DFS_recursive(G, neighbor, visited)
```

#### Pseudocode (iterative with stack)

```
DFS_iterative(G, start):
    stack = [start]
    visited = set()
    
    while stack not empty:
        node = stack.pop()
        
        if node not in visited:
            visited.add(node)
            
            for neighbor in G.neighbors(node):
                if neighbor not in visited:
                    stack.push(neighbor)
    
    return visited
```

#### Visual Example

For a graph with starting node A having neighbors B and C:
- Visit A
- Choose B and recursively explore its entire branch before returning
- Return to A and explore C and its branch

### 5.1 DFS Implementation (recursive)


In [21]:
%%time

def dfs_recursive(G, node, visited, order, end=None, parent=None, path=None):
    """
    Recursive DFS
    
    Parameters:
    - G: NetworkX graph
    - node: current node
    - visited: set of visited nodes
    - order: list of visit order
    - end: destination node (optional)
    - parent: parent dictionary to reconstruct path
    - path: current path (to return found path)
    """
    visited.add(node)
    order.append(node)
    
    if path is None:
        path = []
    path.append(node)
    
    # If we found the target node
    if end is not None and node == end:
        return True, path
    
    # Traverse neighbors
    for neighbor in G.neighbors(node):
        if neighbor not in visited:
            if parent is not None:
                parent[neighbor] = node
            found, found_path = dfs_recursive(G, neighbor, visited, order, end, parent, path.copy())
            if found:
                return True, found_path
    
    return False, None

def dfs(G, start, end=None):
    """
    Depth-first search (DFS)
    
    Parameters:
    - G: NetworkX graph
    - start: starting node
    - end: destination node (optional)
    
    Returns:
    - visited: set of visited nodes
    - path: path to 'end' if specified, None otherwise
    - order: order of node visits
    """
    visited = set()
    order = []
    parent = {start: None}
    
    found, path = dfs_recursive(G, start, visited, order, end, parent, None)
    
    if end is not None:
        return visited, path if found else None, order
    
    return visited, None, order

# Test DFS
print("Running DFS...")
visited_dfs, path_dfs, order_dfs = dfs(G, start_node, end_node)

print(f"Number of visited nodes: {len(visited_dfs)}")
print(f"Path length found: {len(path_dfs) if path_dfs else 'No path'}")

if path_dfs:
    print(f"Path (first 10 nodes): {path_dfs[:10]}...")
    print(f"Path (last 10 nodes): ...{path_dfs[-10:]}")


Running DFS...
Number of visited nodes: 2984
Path length found: 2268
Path (first 10 nodes): [125730, 12179625841, 4467439444, 21112878, 331861643, 76291820, 2841627109, 84791219, 1766131439, 1089076082]...
Path (last 10 nodes): ...[27286561, 27235582, 442309, 27235547, 27286553, 442319, 27233766, 27233754, 5204784909, 27235559]
CPU times: user 54.9 ms, sys: 5.63 ms, total: 60.6 ms
Wall time: 61.8 ms


### 5.2 DFS Implementation (iterative with stack)


In [22]:
%%time

def dfs_iterative(G, start, end=None):
    """
    Iterative DFS using a stack
    
    Parameters:
    - G: NetworkX graph
    - start: starting node
    - end: destination node (optional)
    
    Returns:
    - visited: set of visited nodes
    - path: path to 'end' if specified, None otherwise
    - order: order of node visits
    """
    visited = set()
    stack = [start]
    order = []
    parent = {start: None}
    
    while stack:
        node = stack.pop()
        
        if node not in visited:
            visited.add(node)
            order.append(node)
            
            # If we found the target node
            if end is not None and node == end:
                # Reconstruct the path
                path = []
                current = end
                while current is not None:
                    path.append(current)
                    current = parent[current]
                return visited, list(reversed(path)), order
            
            # Add neighbors to stack (in reverse order for consistency)
            neighbors = list(G.neighbors(node))
            neighbors.reverse()
            for neighbor in neighbors:
                if neighbor not in visited:
                    stack.append(neighbor)
                    if neighbor not in parent:
                        parent[neighbor] = node
    
    # If end was specified but not found
    if end is not None:
        return visited, None, order
    
    return visited, None, order

# Test iterative DFS
print("Running iterative DFS...")
visited_dfs_iter, path_dfs_iter, order_dfs_iter = dfs_iterative(G, start_node, end_node)

print(f"Number of visited nodes: {len(visited_dfs_iter)}")
print(f"Path length found: {len(path_dfs_iter) if path_dfs_iter else 'No path'}")

if path_dfs_iter:
    print(f"Path (first 10 nodes): {path_dfs_iter[:10]}...")
    print(f"Path (last 10 nodes): ...{path_dfs_iter[-10:]}")


Running iterative DFS...
Number of visited nodes: 2984
Path length found: 573
Path (first 10 nodes): [125730, 12179625841, 4467439444, 21112878, 2841627109, 84791219, 1766131439, 1089076082, 1089076107, 1089076163]...
Path (last 10 nodes): ...[25075891, 255028808, 27286561, 27235582, 442309, 27235547, 27286553, 442319, 27233766, 27235559]
CPU times: user 5.63 ms, sys: 132 µs, total: 5.77 ms
Wall time: 5.75 ms


### 5.3 DFS Visualization


In [12]:
# Create a subgraph containing nodes visited by DFS
if path_dfs_iter:
    nodes_in_path_dfs = set()
    for i in range(len(path_dfs_iter) - 1):
        nodes_in_path_dfs.add(path_dfs_iter[i])
        nodes_in_path_dfs.add(path_dfs_iter[i+1])
        for neighbor in G.neighbors(path_dfs_iter[i]):
            nodes_in_path_dfs.add(neighbor)
    
    G_dfs = G.subgraph(list(nodes_in_path_dfs)[:500]).copy()
    
    # Create a color dictionary to show visit order
    node_colors_dfs = {}
    for i, node in enumerate(order_dfs_iter[:100]):
        if node in path_dfs_iter:
            node_colors_dfs[node] = 'red'
        else:
            intensity = int(255 * (i / min(100, len(order_dfs_iter))))
            node_colors_dfs[node] = f'rgb({intensity}, 200, {255-intensity})'
    
    fig = plot_graph_plotly(
        G_dfs, 
        title="DFS Traversal - Paris Road Network",
        highlight_nodes=[start_node, end_node],
        highlight_path=path_dfs_iter,
        node_colors=node_colors_dfs
    )
    fig.show()
    
    print(f"DFS visualization: {len(G_dfs.nodes())} nodes displayed")
else:
    print("No path found with DFS")


DFS visualization: 500 nodes displayed


### 5.4 DFS with NetworkX (built-in function)


In [23]:
%%time
# Using NetworkX's built-in DFS function
try:
    dfs_edges_nx = list(nx.dfs_edges(G, source=start_node))
    dfs_tree_nx = nx.dfs_tree(G, source=start_node)
    
    print(f"NetworkX DFS - Number of edges explored: {len(dfs_edges_nx)}")
    print(f"NetworkX DFS - Tree size: {len(dfs_tree_nx.nodes())} nodes")
    
except Exception as e:
    print(f"Error with NetworkX DFS: {e}")


NetworkX DFS - Number of edges explored: 9433
NetworkX DFS - Tree size: 9434 nodes
CPU times: user 106 ms, sys: 8.18 ms, total: 114 ms
Wall time: 128 ms


## 6. BFS vs DFS Comparison

| Characteristic | BFS | DFS |
|----------------|-----|-----|
| Structure used | Queue (FIFO) | Stack (LIFO) |
| Traversal type | Level by level | Depth-first |
| Shortest path (unweighted graph) | Yes | Not guaranteed |
| Memory usage | Higher (keeps all levels) | Lower |
| Time complexity | O(V + E) | O(V + E) |


In [14]:
# Compare results
print("=== BFS vs DFS Comparison ===\n")

print(f"BFS:")
print(f"  - Visited nodes: {len(visited_bfs)}")
print(f"  - Path length: {len(path_bfs) if path_bfs else 'N/A'}")

print(f"\nDFS:")
print(f"  - Visited nodes: {len(visited_dfs_iter)}")
print(f"  - Path length: {len(path_dfs_iter) if path_dfs_iter else 'N/A'}")

if path_bfs and path_dfs_iter:
    print(f"\nBFS finds a path of {len(path_bfs)} nodes")
    print(f"DFS finds a path of {len(path_dfs_iter)} nodes")
    print(f"Difference: {abs(len(path_bfs) - len(path_dfs_iter))} nodes")
    print(f"\nNote: On an unweighted graph, BFS guarantees the shortest path in number of nodes.")


=== BFS vs DFS Comparison ===

BFS:
  - Visited nodes: 3774
  - Path length: 37

DFS:
  - Visited nodes: 2984
  - Path length: 573

BFS finds a path of 37 nodes
DFS finds a path of 573 nodes
Difference: 536 nodes

Note: On an unweighted graph, BFS guarantees the shortest path in number of nodes.


## 7. Shortest Path Algorithms

### 7.1 Dijkstra (shortest path with weights)

Dijkstra's algorithm finds the shortest path in a weighted graph.


In [15]:
# Check if the graph has length/distance attributes
sample_edge = list(G.edges(data=True))[0]
print(f"Sample edge with attributes: {sample_edge}")

# Use length if available, otherwise use a calculated distance
if 'length' in sample_edge[2]:
    weight = 'length'
    print("\nUsing 'length' attribute for weights")
elif 'weight' in sample_edge[2]:
    weight = 'weight'
    print("\nUsing 'weight' attribute for weights")
else:
    # Calculate geodesic distance if lon/lat available
    weight = None
    print("\nCalculating geodesic distance for weights")
    
    def calculate_distance(u, v):
        """Calculate geodesic distance between two nodes"""
        try:
            if 'x' in G.nodes[u] and 'y' in G.nodes[u]:
                x1, y1 = G.nodes[u]['x'], G.nodes[u]['y']
                x2, y2 = G.nodes[v]['x'], G.nodes[v]['y']
            elif 'lon' in G.nodes[u] and 'lat' in G.nodes[u]:
                x1, y1 = G.nodes[u]['lon'], G.nodes[u]['lat']
                x2, y2 = G.nodes[v]['lon'], G.nodes[v]['lat']
            else:
                return 1.0  # Default distance
            
            # Simple euclidean distance (or use geopy for real distance)
            return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        except:
            return 1.0
    
    # Add calculated weights to graph
    for u, v, d in G.edges(data=True):
        d['calculated_length'] = calculate_distance(u, v)
    weight = 'calculated_length'


Sample edge with attributes: (125730, 12179625841, {'osmid': 31093720, 'highway': 'residential', 'lanes': '2', 'maxspeed': '30', 'name': 'Rue Louis Lumière', 'oneway': False, 'reversed': True, 'length': np.float64(4.347746894370387)})

Using 'length' attribute for weights


In [16]:
# Calculate shortest path with Dijkstra
try:
    print("Calculating shortest path with Dijkstra...")
    path_dijkstra = nx.shortest_path(G, source=start_node, target=end_node, weight=weight, method='dijkstra')
    length_dijkstra = nx.shortest_path_length(G, source=start_node, target=end_node, weight=weight, method='dijkstra')
    
    print(f"Dijkstra path - Length: {len(path_dijkstra)} nodes")
    print(f"Total distance: {length_dijkstra:.2f} meters" if weight else f"Total distance: {length_dijkstra:.2f}")
    print(f"Path (first 10 nodes): {path_dijkstra[:10]}...")
    print(f"Path (last 10 nodes): ...{path_dijkstra[-10:]}")
    
except Exception as e:
    print(f"Error with Dijkstra: {e}")
    path_dijkstra = None
    length_dijkstra = None


Calculating shortest path with Dijkstra...
Dijkstra path - Length: 125 nodes
Total distance: 9343.17 meters
Path (first 10 nodes): [125730, 1089076182, 1089076216, 1766131427, 10299440575, 84791232, 134521832, 2405282107, 2405282109, 134521836]...
Path (last 10 nodes): ...[27234093, 1107432316, 5095554625, 5095554623, 27233726, 1369651515, 367908, 27233754, 5204784909, 27235559]


### 7.2 A* (A-star) - Heuristic for optimization

#### 7.2.0 A* Algorithm Theory

The **A*** algorithm (pronounced "A-star") is an improvement of Dijkstra's algorithm that uses a **heuristic function** to guide the search towards the goal, thus reducing the number of nodes explored.

#### Principle of Operation

A* combines two pieces of information to decide which node to explore next:

1. **g(n)**: Actual cost of the path from the start node to node n
2. **h(n)**: Heuristic estimate of the remaining cost from node n to the goal

The evaluation function is: **f(n) = g(n) + h(n)**

The algorithm always explores the node with the smallest f(n) value.

#### Heuristic Function

A good heuristic must be:

- **Admissible**: Never overestimates the actual cost to the goal (h(n) ≤ actual cost)
- **Consistent/Monotone**: For any node n and its successor n', h(n) ≤ c(n,n') + h(n')

#### Important Properties

- **Optimality**: If the heuristic is admissible, A* guarantees finding the optimal path
- **Efficiency**: Generally explores fewer nodes than Dijkstra, especially with a good heuristic
- **Time complexity**: O((V + E) log V) in worst case (like Dijkstra), but often better in practice
- **Space complexity**: O(V) to store explored nodes

#### Comparison with Dijkstra

| Characteristic | Dijkstra | A* |
|----------------|----------|-----|
| Heuristic | No | Yes |
| Exploration | Uniform (in circles) | Goal-directed |
| Nodes explored | More | Fewer (with good heuristic) |
| Optimality | Yes | Yes (if heuristic admissible) |

#### Common Heuristics

1. **Euclidean distance**: h(n) = euclidean distance between n and goal (for geographic graphs)
2. **Manhattan distance**: h(n) = |x_n - x_goal| + |y_n - y_goal| (for grids)
3. **Chebyshev distance**: For diagonal movements allowed

#### Applications

- Video games: character pathfinding
  - Examples: Starcraft, Age of Empires, strategy games
- Robotics: trajectory planning
- GPS navigation (variants with constraints)
- Artificial intelligence: puzzle solving, state space search

#### Pseudocode

```
A_star(G, start, goal, heuristic):
    open_set = [(heuristic(start, goal), start)]  # (f, node)
    g_score = {node: ∞ for node in G.nodes}
    g_score[start] = 0
    f_score = {node: ∞ for node in G.nodes}
    f_score[start] = heuristic(start, goal)
    parent = {start: None}
    
    while open_set not empty:
        current = open_set.extract_min()  # node with smallest f
        
        if current == goal:
            return reconstruct_path(parent, current)
        
        for neighbor in G.neighbors(current):
            tentative_g = g_score[current] + weight(current, neighbor)
            
            if tentative_g < g_score[neighbor]:
                parent[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
                open_set.insert_or_update((f_score[neighbor], neighbor))
    
    return None  # No path found
```

The A* algorithm is an improvement of Dijkstra that uses a heuristic to guide the search.


In [17]:
# Heuristic function for A* (euclidean distance to goal)
def heuristic(u, v):
    """Heuristic: euclidean distance between two nodes"""
    try:
        if 'x' in G.nodes[u] and 'y' in G.nodes[u]:
            x1, y1 = G.nodes[u]['x'], G.nodes[u]['y']
            x2, y2 = G.nodes[v]['x'], G.nodes[v]['y']
        elif 'lon' in G.nodes[u] and 'lat' in G.nodes[u]:
            x1, y1 = G.nodes[u]['lon'], G.nodes[u]['lat']
            x2, y2 = G.nodes[v]['lon'], G.nodes[v]['lat']
        else:
            return 0
        
        return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    except:
        return 0

# Calculate shortest path with A*
try:
    print("Calculating shortest path with A*...")
    path_astar = nx.astar_path(G, source=start_node, target=end_node, weight=weight, heuristic=heuristic)
    length_astar = nx.astar_path_length(G, source=start_node, target=end_node, weight=weight, heuristic=heuristic)
    
    print(f"A* path - Length: {len(path_astar)} nodes")
    print(f"Total distance: {length_astar:.2f} meters" if weight else f"Total distance: {length_astar:.2f}")
    print(f"Path (first 10 nodes): {path_astar[:10]}...")
    print(f"Path (last 10 nodes): ...{path_astar[-10:]}")
    
except Exception as e:
    print(f"Error with A*: {e}")
    path_astar = None
    length_astar = None


Calculating shortest path with A*...
A* path - Length: 125 nodes
Total distance: 9343.17 meters
Path (first 10 nodes): [125730, 1089076182, 1089076216, 1766131427, 10299440575, 84791232, 134521832, 2405282107, 2405282109, 134521836]...
Path (last 10 nodes): ...[27234093, 1107432316, 5095554625, 5095554623, 27233726, 1369651515, 367908, 27233754, 5204784909, 27235559]


### 7.3 Comparison of shortest path algorithms


In [18]:
import time

print("=== Algorithm Comparison ===\n")

# Build algorithms based on weight availability
algorithms = {
    'BFS (unweighted)': lambda: nx.shortest_path(G, start_node, end_node, method='bfs')
}

if weight:
    algorithms['Dijkstra'] = lambda: nx.shortest_path(G, start_node, end_node, weight=weight, method='dijkstra')
    algorithms['A*'] = lambda: nx.astar_path(G, start_node, end_node, weight=weight, heuristic=heuristic)
else:
    # If no weights, Dijkstra is equivalent to BFS
    algorithms['Dijkstra (no weights)'] = lambda: nx.shortest_path(G, start_node, end_node, method='dijkstra')

results = {}
for name, func in algorithms.items():
    try:
        start_time = time.time()
        path = func()
        elapsed = time.time() - start_time
        length = len(path)
        
        if weight:
            path_length = nx.shortest_path_length(G, start_node, end_node, weight=weight)
            results[name] = {
                'path': path,
                'node_count': length,
                'distance': path_length,
                'time': elapsed
            }
        else:
            results[name] = {
                'path': path,
                'node_count': length,
                'distance': None,
                'time': elapsed
            }
        print(f"{name}:")
        print(f"  - Time: {elapsed*1000:.2f} ms")
        print(f"  - Number of nodes: {length}")
        if weight:
            print(f"  - Distance: {path_length:.2f}")
        print()
    except Exception as e:
        print(f"{name}: Error - {e}\n")


=== Algorithm Comparison ===

BFS (unweighted): Error - method not supported: bfs

Dijkstra:
  - Time: 45.70 ms
  - Number of nodes: 125
  - Distance: 9343.17

A*:
  - Time: 73.72 ms
  - Number of nodes: 125
  - Distance: 9343.17



### 7.4 Visualization of shortest paths


In [19]:
# Create a visualization comparing different paths
if path_dijkstra:
    # Create a subgraph containing the paths
    all_path_nodes = set()
    for path in [path_bfs, path_dfs_iter, path_dijkstra, path_astar]:
        if path:
            all_path_nodes.update(path)
            # Add some neighbors for context
            for node in path[:50]:  # Limit for performance
                for neighbor in list(G.neighbors(node))[:3]:
                    all_path_nodes.add(neighbor)
    
    G_paths = G.subgraph(list(all_path_nodes)[:1000]).copy()
    
    # Visualization with different paths
    fig = go.Figure()
    
    # Get coordinates (longitude, latitude)
    pos = {}
    for node in G_paths.nodes():
        if 'x' in G_paths.nodes[node] and 'y' in G_paths.nodes[node]:
            pos[node] = (G_paths.nodes[node]['x'], G_paths.nodes[node]['y'])
        elif 'lon' in G_paths.nodes[node] and 'lat' in G_paths.nodes[node]:
            pos[node] = (G_paths.nodes[node]['lon'], G_paths.nodes[node]['lat'])
        else:
            pos[node] = (node, 0)
    
    # Calculate map center based on paths
    path_coords = []
    for path in [path_dijkstra, path_astar, path_bfs]:
        if path:
            for node in path:
                if node in pos and isinstance(pos[node][0], (int, float)) and isinstance(pos[node][1], (int, float)):
                    path_coords.append(pos[node])
    
    if path_coords:
        center_lon = sum(c[0] for c in path_coords) / len(path_coords)
        center_lat = sum(c[1] for c in path_coords) / len(path_coords)
    else:
        center_lon, center_lat = 2.3522, 48.8566  # Paris by default
    
    # Graph edges - grouped in a single trace for performance
    edge_lons = []
    edge_lats = []
    for edge in G_paths.edges():
        if edge[0] in pos and edge[1] in pos:
            lon0, lat0 = pos[edge[0]]
            lon1, lat1 = pos[edge[1]]
            if (isinstance(lon0, (int, float)) and isinstance(lat0, (int, float)) and
                isinstance(lon1, (int, float)) and isinstance(lat1, (int, float))):
                edge_lons.extend([lon0, lon1, None])
                edge_lats.extend([lat0, lat1, None])
    
    if edge_lons and edge_lats:
        fig.add_trace(go.Scattermap(
            mode='lines',
            lon=edge_lons,
            lat=edge_lats,
            line=dict(width=0.5, color='lightgray'),
            hoverinfo='none',
            name='Road network',
            showlegend=True,
            legendgroup='network'
        ))
    
    # Dijkstra path
    if path_dijkstra:
        path_lons = [pos[node][0] for node in path_dijkstra if node in pos and isinstance(pos[node][0], (int, float))]
        path_lats = [pos[node][1] for node in path_dijkstra if node in pos and isinstance(pos[node][1], (int, float))]
        if path_lons and path_lats:
            fig.add_trace(go.Scattermap(
                mode='lines+markers',
                lon=path_lons,
                lat=path_lats,
                line=dict(width=4, color='blue'),
                marker=dict(size=6, color='blue', opacity=0.7),
                name=f'Dijkstra ({len(path_dijkstra)} nodes)',
                showlegend=True,
                legendgroup='paths'
            ))
    
    # A* path
    if path_astar:
        path_lons = [pos[node][0] for node in path_astar if node in pos and isinstance(pos[node][0], (int, float))]
        path_lats = [pos[node][1] for node in path_astar if node in pos and isinstance(pos[node][1], (int, float))]
        if path_lons and path_lats:
            fig.add_trace(go.Scattermap(
                mode='lines+markers',
                lon=path_lons,
                lat=path_lats,
                line=dict(width=4, color='green'),
                marker=dict(size=6, color='green', opacity=0.7),
                name=f'A* ({len(path_astar)} nodes)',
                showlegend=True,
                legendgroup='paths'
            ))
    
    # BFS path
    if path_bfs:
        path_lons = [pos[node][0] for node in path_bfs if node in pos and isinstance(pos[node][0], (int, float))]
        path_lats = [pos[node][1] for node in path_bfs if node in pos and isinstance(pos[node][1], (int, float))]
        if path_lons and path_lats:
            fig.add_trace(go.Scattermap(
                mode='lines+markers',
                lon=path_lons,
                lat=path_lats,
                line=dict(width=3, color='red'),
                marker=dict(size=5, color='red', opacity=0.7),
                name=f'BFS ({len(path_bfs)} nodes)',
                showlegend=True,
                legendgroup='paths'
            ))
    
    # Start and end points
    if start_node in pos and end_node in pos:
        start_lon, start_lat = pos[start_node]
        end_lon, end_lat = pos[end_node]
        if (isinstance(start_lon, (int, float)) and isinstance(start_lat, (int, float)) and
            isinstance(end_lon, (int, float)) and isinstance(end_lat, (int, float))):
            fig.add_trace(go.Scattermap(
                mode='markers',
                lon=[start_lon],
                lat=[start_lat],
                marker=dict(size=18, color='orange'),
                name='Start',
                showlegend=True,
                legendgroup='points'
            ))
            fig.add_trace(go.Scattermap(
                mode='markers',
                lon=[end_lon],
                lat=[end_lat],
                marker=dict(size=18, color='purple'),
                name='End',
                showlegend=True,
                legendgroup='points'
            ))
    
    fig.update_layout(
        title="Shortest Paths Comparison - Paris",
        showlegend=True,
        mapbox=dict(
            style="open-street-map",
            center=dict(lon=center_lon, lat=center_lat),
            zoom=12,
            bearing=0,
            pitch=0
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        height=800
    )
    
    fig.show()


## 8. Summary

### Key Points to Remember:

1. **BFS (Breadth-First Search)** :
   - Explores level by level
   - Guarantees shortest path in number of nodes (unweighted graph)
   - Uses a queue (FIFO)

2. **DFS (Depth-First Search)** :
   - Explores in depth before backtracking
   - Does not guarantee shortest path
   - Uses a stack (LIFO) or recursion

3. **Dijkstra** :
   - Finds shortest path in a weighted graph
   - Requires positive weights
   - Complexity: O((V + E) log V) with good implementation

4. **A*** :
   - Improvement of Dijkstra with heuristic
   - Faster than Dijkstra in most cases
   - Optimal if heuristic is admissible (never overestimates)

### Practical Applications:
- GPS navigation (routing)
- Social networks (finding connections)
- Artificial intelligence (games, planning)
- Network analysis (Internet, transportation)
